In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Online_Retail.csv")
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom


In [3]:
df.dropna(inplace=True, axis=0)

In [4]:
df.drop(columns=['InvoiceNo','StockCode','CustomerID'], axis=1, inplace=True)

In [5]:
drop_quantity = df[(df['Quantity'] < 1) | (df['Quantity'] > 40)]
df = df.drop(drop_quantity.index, axis=0)


# drop rows for UnitPrice less than 1 and greater than 30
drop_UnitPrice = df[(df['UnitPrice'] < 1) | (df['UnitPrice'] > 30)]
df = df.drop(drop_UnitPrice.index, axis=0)

df["UnitPrice"] = df['UnitPrice'].astype(int)
            

# drop rows for Description < 300
df['Description']=df['Description'].str.strip()
drop_Description = df['Description'].value_counts()[df['Description'].value_counts() < 300]
drop_Description_index=drop_Description.index

df = df.drop(df[df['Description'].isin(drop_Description_index)].index, axis=0)

df.shape

(205776, 5)

In [6]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Day']=df['InvoiceDate'].dt.day
df['Month']=df['InvoiceDate'].dt.month
df['Year']=df['InvoiceDate'].dt.year - 2000

df = df.drop(['InvoiceDate'],axis=1)

date_col = ['Day', 'Month', 'Year']
for i in date_col:
    df[i] = df[i].astype(int)
    
df.head(3)

,Description,Quantity,UnitPrice,Country,Day,Month,Year
0,WHITE HANGING HEART T-LIGHT HOLDER,6,2,United Kingdom,1,12,10
1,WHITE METAL LANTERN,6,3,United Kingdom,1,12,10
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,3,United Kingdom,1,12,10


In [7]:
df['Total_price'] = df['Quantity'] * df['UnitPrice']
df.head(4)

,Description,Quantity,UnitPrice,Country,Day,Month,Year,Total_price
0,WHITE HANGING HEART T-LIGHT HOLDER,6,2,United Kingdom,1,12,10,12
1,WHITE METAL LANTERN,6,3,United Kingdom,1,12,10,18
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,3,United Kingdom,1,12,10,18
4,RED WOOLLY HOTTIE WHITE HEART.,6,3,United Kingdom,1,12,10,18


In [8]:
top_n_values = 100
#Get the top N most frequent values in the column and convert into list
top_values = df['Description'].value_counts().nlargest(top_n_values).index.tolist()

# Filter the column to include only the top N values
filtered_column = df['Description'].where(df['Description'].isin(top_values), other='Others')

df['Description'] = filtered_column.values

In [9]:
df.shape

(205776, 8)

In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor

In [11]:
df = pd.get_dummies(df, prefix='', prefix_sep='', dtype=int, drop_first=True)
df.head(5)

,Quantity,UnitPrice,Day,Month,Year,Total_price,ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE PINK,ALARM CLOCK BAKELIKE RED,ANTIQUE SILVER T-LIGHT GLASS,...,RSA,Saudi Arabia,Singapore,Spain,Sweden,Switzerland,USA,United Arab Emirates,United Kingdom,Unspecified
0,6,2,1,12,10,12,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,6,3,1,12,10,18,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,6,3,1,12,10,18,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,6,3,1,12,10,18,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,2,7,1,12,10,14,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [12]:
df.columns

Index(['Quantity', 'UnitPrice', 'Day', 'Month', 'Year', 'Total_price',
       'ALARM CLOCK BAKELIKE GREEN', 'ALARM CLOCK BAKELIKE PINK',
       'ALARM CLOCK BAKELIKE RED', 'ANTIQUE SILVER T-LIGHT GLASS',
       ...
       'RSA', 'Saudi Arabia', 'Singapore', 'Spain', 'Sweden', 'Switzerland',
       'USA', 'United Arab Emirates', 'United Kingdom', 'Unspecified'],
      dtype='object', length=143)

In [13]:
x = df.drop("Total_price", axis=1)
y = df.Total_price
x_train, x_test, y_train, y_test = train_test_split(x,y)

In [14]:
x_train.shape

(154332, 142)

In [16]:
lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
print("r2_score : ", r2_score(y_test, y_pred))
print("mse : ", mean_squared_error(y_test, y_pred))


r2_score :  0.589492694156659
mse :  163.03040994996613


In [17]:
dtr = DecisionTreeRegressor()
dtr.fit(x_train, y_train)
y_pred = dtr.predict(x_test)
print("r2_score : ", r2_score(y_test, y_pred))
print("mse : ", mean_squared_error(y_test, y_pred))


r2_score :  0.9977016898528736
mse :  0.9127595054816888
